In [130]:
import pandas as pd
import numpy as np
import re

In [131]:
real_column_list = ['レコードID','講演会参加日','対象医薬品名','販売元','薬効分類コード','薬効分類名','適応',
               '開催都道府県','開催市区町村','講演者','講演者の所属施設','座長', '座長の所属施設','講演テーマ','ハッシュ値']

web_cowlumn_list = ['レコードID','講演会視聴日','視聴時間帯','対象医薬品名','販売元','薬効分類コード','薬効分類名','適応',
                  '講演者','講演者の所属施設', '座長', '座長の所属施設','講演テーマ','ハッシュ値']

# 縦マージ

## ①不要列の削除　②座長用データの複製

In [132]:
df1 = pd.read_csv('whitedata.csv',encoding ='cp932',usecols = real_column_list)
df2 = pd.read_csv('whitedata.csv',encoding ='cp932',usecols = real_column_list)

## ③フラグ追加

In [133]:
df1['フラグ'] ='講演者'
df2['フラグ'] ='座長'

## ⑤座長情報，座長所属施設情報の移植

In [134]:
df2['講演者']= df1['座長']
df2['講演者の所属施設'] = df1['座長の所属施設']

In [135]:
df = pd.concat([df1,df2],axis = 0)

## ④フラグ&レコード（ユニークID）作成

In [136]:
df['フラグ&レコード'] = df['フラグ'] + df['レコードID'].astype(str)

## ⑥「座長」,「座長所属施設」列削除

In [137]:
df.drop(['座長','座長の所属施設'],inplace = True,axis =1)

In [138]:
df.head()

,レコードID,講演会参加日,対象医薬品名,販売元,薬効分類コード,薬効分類名,適応,講演テーマ,ハッシュ値,開催都道府県,開催市区町村,講演者,講演者の所属施設,フラグ,フラグ&レコード
0,1,3月30日,メンゾール,武田,CA,糖尿病,がん,がん,931431890,東京,仙台,矢作 直也,筑波大学,講演者,講演者1
1,2,3月30日,メンゾール,武田,CA,糖尿病,がん,がん,931431890,東京,仙台,鈴木 浩明,筑波大学 医学医療系,講演者,講演者2
2,3,3月30日,メンゾール,武田,CA,糖尿病,がん,がん,931431890,東京,仙台,特定できず,特定できず,講演者,講演者3
3,4,3月30日,メンゾール,武田,CA,糖尿病,がん,がん,931431890,東京,仙台,特定できず,NaN,講演者,講演者4
4,5,3月30日,メンゾール,武田,CA,糖尿病,がん,がん,931431890,東京,仙台,志鎌 明人,茨城県立中央病院,講演者,講演者5


## ⑦列名の並び替え

In [139]:
df = df.ix[:,['フラグ','レコードID','フラグ&レコード','講演会参加日','対象医薬品名','販売元','薬効分類コード','薬効分類名','適応',
               '開催都道府県','開催市区町村','講演者','講演者の所属施設','講演テーマ','ハッシュ値']]

/Users/shintomi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [140]:
df.head()

,フラグ,レコードID,フラグ&レコード,講演会参加日,対象医薬品名,販売元,薬効分類コード,薬効分類名,適応,開催都道府県,開催市区町村,講演者,講演者の所属施設,講演テーマ,ハッシュ値
0,講演者,1,講演者1,3月30日,メンゾール,武田,CA,糖尿病,がん,東京,仙台,矢作 直也,筑波大学,がん,931431890
1,講演者,2,講演者2,3月30日,メンゾール,武田,CA,糖尿病,がん,東京,仙台,鈴木 浩明,筑波大学 医学医療系,がん,931431890
2,講演者,3,講演者3,3月30日,メンゾール,武田,CA,糖尿病,がん,東京,仙台,特定できず,特定できず,がん,931431890
3,講演者,4,講演者4,3月30日,メンゾール,武田,CA,糖尿病,がん,東京,仙台,特定できず,NaN,がん,931431890
4,講演者,5,講演者5,3月30日,メンゾール,武田,CA,糖尿病,がん,東京,仙台,志鎌 明人,茨城県立中央病院,がん,931431890


#### indexの振り直し

In [148]:
df = df.reset_index()

## ⑧エクセルファイルに書き出し

In [149]:
df.to_excel('tate_marge.xlsx')

# 整備

## ①各種変数定義,製薬会社マスタ取得,いない忘れたマスタ取得，過去名寄せデータ取得

### 1.各種変数定義

In [144]:
sensei = ['　',' ','先生','医師','教授','Dr','院長','演者',':','他','ほか','医者','部長','助教','准教授']#検索用に医師名（施設名）から取り除くリスト

irekae =['院','センター','クリニック','科','大学']#検索用医師名と施設名を入れ替えるリスト

MR =['-','MR','担当者','社員','メーカー']#削除フラグ用

past_column = ['フラグ&レコード','回答者DCFコード','回答者DCF医師名','回答者DCF施設コード',
                                                           '回答者DCF施設名','データ連結週','作業者', '作業日']#<-要変更

rename_list = ["公立大学法人", "公益財団法人", "社会福祉法人", "社会医療法人", "国立研究開発法人", "一般社団法人", "独立行政法人",
               "医療法人", "社団法人", "社団", "学校法人", "学校法人", "国立大学法人", "医学部附属病院", 
               "医学部", "附属病院", "附属", "財団","（医）","（医社）","国立病院機構"]#検索用施設名から除くリスト

### 2.製薬会社一覧取得

In [145]:
df_seiyaku = pd.read_csv('seiyaku.csv',encoding ='cp932')

#### 製薬の一列目を半角小文字に統一

In [ ]:
df_seiyaku['一列目'] = df_seiyaku['一列目']str.lower()

# 3.!!!いない，忘れたマスタを取得とは？

### 4.前週までに名寄せしたものを取得

In [63]:
df_past = pd.read_excel('kakonayose.xlsx' ,usecols=past_column)

## ②過去名寄せデータとの比較，名寄せられていないデータのみ取得

In [158]:
df_new = df[~df['フラグ&レコード'].isin(df_past['フラグ&レコード'])]

### *連結週，作業者・作業日の作成

## ③検索用医師名，検索用施設名の作成

### 1.大学以降の余分な成分を取り除く，大学前に余分な成分が含まれている場合もrename_listを参照して取り除く

In [155]:
df['講演者の所属施設'] = df['講演者の所属施設'].astype(str)
df['講演者'] = df['講演者'].astype(str)

In [156]:
for i,name in enumerate(df['講演者の所属施設']):
    df.loc[i,'検索用施設名_正規化'] = re.sub(r'(.*大学).*', r'\1', name)
for i,name in enumerate(df['講演者']):
    df.loc[i,'検索用医師名_正規化'] = re.sub(r'(.*大学).*', r'\1', name)
for target_char in rename_list:
    df['検索用施設名_正規化'] = df['検索用施設名_正規化'].str.replace(target_char, '')
for target_char in rename_list:
    df['検索用医師名_正規化'] = df['検索用医師名_正規化'].str.replace(target_char, '')

### 2.正規化ファイルの読み込み,辞書ファイルを作成

In [157]:
import csv
dict = {}
with open('seikika.csv',encoding='cp932') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        dict[row[0]] =row[1]

#for key, value in dict.items():
    #print('{}: {}'.format(key, value))  

#### 正規化ファイルを用いて講演者の所属施設と講演者を正規化

In [80]:
df['検索用施設名_正規化'] = df['講演者の所属施設'].replace(dict)
df['検索用医師名_正規化'] = df['講演者'].replace(dict)

### 3.検索用医師名から余分な要素を取り除く

### 4.検索用医師名と検索用施設名が入れ替わっているものを入れ替える

## ④各種フラグ立て

### 4444444のフラグ

### 5555555のフラグ

### 6666666のフラグ

### 7777777のフラグ

### 8888888のフラグ